In [1]:
import sys
sys.path.append('/Users/ManuelSerraNunes/Desktop/Tecnico_5_1S/IA/Artificial_Intelligence_IST/mini_project1')
#sys.path.append(r'C:\Users\Carlos\Desktop\IST\IST 5ºano\IA\Artificial_Intelligence_IST\mini_project1')
#sys.path.append('/Users/simaomoraessarmento/Dropbox/IST/S1/IA/Artificial_Intelligence_IST/mini_project1')
from go import *

## Completed

In [2]:
#file_path = r"C:\Users\Carlos\Desktop\IST\IST 5ºano\IA\Artificial_Intelligence_IST\mini_project1\examples\example5.txt"
file_path = "/Users/ManuelSerraNunes/Desktop/Tecnico_5_1S/IA/Artificial_Intelligence_IST/mini_project1/examples/test05.txt"
#file_path = "/Users/simaomoraessarmento/Dropbox/IST/S1/IA/Artificial_Intelligence_IST/mini_project1/examples/example2.txt"
file = open(file_path)


game = Game()
s = game.load_board(file)
act = game.actions(s)
#game.terminal_test(s)


In [4]:
game.utility(s,1)

-0.3333333333333333

In [4]:
a = {'b': 2}

In [3]:
alphabeta_cutoff_search(s, game, d=4, cutoff_test=None, eval_fn=None)


0.3333333333333333 (2, 2, 2)
0.3333333333333333 (2, 3, 5)
1 (2, 5, 3)
1 (2, 2, 5)
1 (2, 1, 4)
1 (2, 2, 1)
1 (2, 1, 1)
1 (2, 2, 4)
1 (2, 5, 5)
1 (2, 3, 1)
1 (2, 1, 5)
1 (2, 4, 2)
1 (2, 5, 1)
1 (2, 1, 2)
1 (2, 4, 5)
1 (2, 2, 3)
1 (2, 5, 2)
1 (2, 1, 3)
1 (2, 4, 1)


(2, 5, 3)

In [20]:
ns = game.result(s, (2,2,3))

In [21]:
s.group_board

[[0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 2, 1, 4, 0],
 [0, 0, 6, 3, 0],
 [0, 0, 0, 3, 0]]

In [22]:
ns.group_board

[[0, 0, 0, 0, 0],
 [0, 0, 8, 0, 0],
 [0, 2, 1, 4, 0],
 [0, 0, 6, 3, 0],
 [0, 0, 0, 3, 0]]

In [23]:
ns.groups[2][6].liberties

{(3, 1), (4, 2)}

In [24]:
game.utility(ns,2)

1

## Function development

### Actions

In [114]:
def actions(s):
    
    """returns list with points of possible actions
    """
    
    actions = set()
    
    for row in range(len(s.group_board)):
        for col in range(len(s.group_board)):
            
            if s.group_board[row][col] == 0:
            
                neighbors = get_neighbors(row, col, s)
                
                for neighbor in neighbors:
                    
                    neighbor_color = neighbor[0]
                    neighbor_liberties = neighbor[1]
                    
                    if neighbor_color == 0:
                        actions.add((s.player, row, col))
                        continue
                    
                    if neighbor_color == s.player and neighbor_liberties > 1: 
                        actions.add((s.player, row, col))
                        continue
                    
                    if neighbor_color != s.player and neighbor_liberties == 1:
                        actions.add((s.player, row, col))
                        continue

    return actions

        
def get_group_color(group_number):
    # podiamos meter a cor do grupo na classe grupo para nao ser preciso isto
    if group_number % 2 == 1:
        return 1
    else:
        return 2

    
def get_neighbors(row, col, s):
    """returns list of tuples (neighbour_color, #liberties)
    """

    board_size = s.size
    neighbors = list()
    coords = [ (row-1,col), (row+1,col), (row,col-1), (row,col+1) ]
    
    for coord in coords:
        row = coord[0]
        col = coord[1]

        if (row >= 0 and row < board_size) and (col >= 0 and col < board_size):
           
            group = s.group_board[row][col] 
            
            if group == 0:
                neighbors.append((0,-1))
            else:
                color = get_group_color(group)
                neighbors.append( (color, s.groups[color][group].n_liberties) )
    
    return neighbors

In [115]:
acts = actions(s)
print(acts)

In [111]:
s.group_board

[[0, 0, 1, 0], [0, 3, 2, 2], [0, 4, 5, 0], [0, 0, 0, 0]]

### Update Liberties

In [ ]:
def update_liberties(row, col, board, group_board, player, groups):
    """
    This function calculates the liberties for the given point,
    and updates the corresponding group information regarding so.
    
    :param row: row point position
    :param col: col point position
    :param board: matrix with board pieces
    :param group_board: matrix with board groups
    :param player: next player to move
    :param groups: dictionary with groups' information
    """
    group = group_board[row][col]
    
    # check all 4 neighbors. If the neighbor is empty (= 0) its a liberty
    
    if row - 1 >= 0:
        if board[row-1][col] == 0:
            groups[player][group].add_liberty((row-1, col))
    if row + 1 < len(board):     
        if board[row+1][col] == 0:
            groups[player][group].add_liberty((row+1, col))
    if col - 1 >= 0:
        if board[row][col-1] == 0:
            groups[player][group].add_liberty((row, col-1))
    if col + 1 < len(board):
        if board[row][col+1] == 0:
            groups[player][group].add_liberty((row, col+1))
    
            
def merge_groups(row, col, player, groups, left_neighbor_group, group_board):
    """
    This function is responsible for merging two connected groups initially
    identified as separated components.
    
    :param row: row point position
    :param col: col point position
    :param player: next player to move
    :param groups: dictionary with groups' information
    :param left_neighbor_group: group to merge
    :param group_board: matrix with board groups
    
    :return board elements to update on the board
    """
    group = group_board[row][col]
    
    # 1) merge elements
    elements_to_merge = groups[player][left_neighbor_group].elements
    groups[player][group].add_element(elements_to_merge)
    
    # 2) merge liberties (liberties count is updated internally)
    liberties_to_merge = groups[player][left_neighbor_group].liberties
    for liberty in liberties_to_merge:
        groups[player][group].add_liberty(liberty)
    
    return elements_to_merge

group_board = [[0] * len(board) for i in range(len(board))]
groups = {1:dict(), 2:dict()}
counters = {1:1, 2:2}

state = State() # vem no cabeçalho

for row in range(len(board)):
    for col in range(len(board)):
        print('\n', row, col)
        player = board[row][col]
        
        # Check if current point is occupied. If not, it just skips to next point
        if board[row][col] != 0:
            """
            Component identification goes as follows:
            1) Look for a group neighbor on the row above. If existent, join group.
            2) Look for a group neighbor on the col on the left. If existent:
                2.1) If neighbor was found both on top and left, then merge left group
                     w/ top group
                2.2) If only left neighbor was found, then join current point to left
                     group.
            3) If no neighbor was found (same player piece in any of the neighboring points)
               then a new group is created
            """
                        
            if row - 1 >= 0:
                top_neighbor = board[row-1][col] 
                top_neighbor_group = group_board[row-1][col] 
    
                if top_neighbor_group != 0 and top_neighbor == player:
                    # updates group board
                    group_board[row][col] = top_neighbor_group
                    # adds element to group structure
                    new_elements = [(row, col)]
                    groups[player][group_board[row][col]].add_element(new_elements)
    
            if col - 1 >= 0:
                left_neighbor = board[row][col-1]
                left_neighbor_group = group_board[row][col-1]
                
                if left_neighbor_group != 0 and left_neighbor == player:
                    # verifies if groups must be merged (condition 2.1)
                    # by checking group_board[row][col] != 0, we ensure a group has been 
                    # attributed before
                    if group_board[row][col] != 0 and left_neighbor_group != group_board[row][col]:
                        # updates left neighbor's group elements
                        elements_to_merge= merge_groups(row, col, player, groups, left_neighbor_group, group_board)
                        # removes old group from dictionary
                        groups[player].pop(left_neighbor_group)
                        # updates group board
                        for old_element_row, old_element_col in elements_to_merge:
                            group_board[old_element_row][old_element_col] = group_board[row][col]
                    
                    # no neighbor on top, just join left neighbor's group (condition 2.2)
                    elif group_board[row][col] == 0:
                        # updates group board
                        group_board[row][col] = left_neighbor_group
                        # adds element to group structure
                        new_elements = [(row, col)]
                        groups[player][group_board[row][col]].add_element(new_elements)
            
            # if no merging neighbors were found, than just creates a new group
            if group_board[row][col] == 0:
                group_board[row][col] = counters[player]
                groups[player][counters[player]] = Group((row, col))
                counters[player] += 2
                
            # Finally, add information about group's liberties to the structure
            update_liberties(row, col, board, group_board, player, groups)
        print(group_board)

In [ ]:
print(groups[1][1].liberties)

In [ ]:
groups[2][2].elements

### Result()

In [23]:
ns = game.result(s, (1,0,0))

In [24]:
s.group_board

[[0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0],
 [0, 3, 0, 5, 0],
 [0, 0, 7, 0, 0],
 [0, 0, 0, 0, 0]]

In [25]:
ns.group_board

[[9, 0, 0, 0, 0],
 [0, 0, 1, 0, 0],
 [0, 3, 0, 5, 0],
 [0, 0, 7, 0, 0],
 [0, 0, 0, 0, 0]]

In [30]:
ns.groups[1][9].liberties

{(0, 1), (1, 0)}

In [32]:
ns.groups[1][9].n_liberties

2